In [8]:
import pandas as pd
import numpy as np
import json
import seaborn as sns
import os
import cv2
from tqdm import tqdm_notebook
from matplotlib import pyplot as plt
from PIL import Image

In [9]:
data_json = '/opt/ml/detection/dataset/train.json
data_json

'/opt/ml/detection/dataset/new_train.json'

In [10]:
with open(data_json, 'r') as outfile:
    data = (json.load(outfile))

df_image = pd.DataFrame(data['images']) #이미지 데이터
df_annotation = pd.DataFrame(data['annotations']) #bbox 
df_categories = pd.DataFrame(data['categories']) #categories

In [11]:
#한번 실행 후 주석처리 해주세요
# os.mkdir('/opt/ml/detection/dataset/new_annotated')

In [12]:
df_image = df_image.sort_values(by=['id'])
df_annotation = df_annotation.sort_values(by=['image_id'])

In [13]:
#BGR 순서
Category_color = pd.DataFrame({'150,100,255', '0,255,255', '255, 200, 0', '255, 100, 0', '100, 150, 50', '0, 100, 255', '102, 0, 0', '180, 220, 145', '200, 255, 255', '255,150,150'}, columns=['color'])
df_categories['color'] = Category_color
df_categories

,id,name,supercategory,color
0,0,General trash,General trash,"0,255,255"
1,1,Paper,Paper,"102, 0, 0"
2,2,Paper pack,Paper pack,"0, 100, 255"
3,3,Metal,Metal,"255,150,150"
4,4,Glass,Glass,"255, 100, 0"
5,5,Plastic,Plastic,"255, 200, 0"
6,6,Styrofoam,Styrofoam,"100, 150, 50"
7,7,Plastic bag,Plastic bag,"150,100,255"
8,8,Battery,Battery,"200, 255, 255"
9,9,Clothing,Clothing,"180, 220, 145"


In [14]:
base_dir = '/opt/ml/detection/dataset/'
for i in tqdm_notebook(range(len(df_image))):
    
    file_name =  df_image.iloc[i]['file_name']

    img = cv2.imread(base_dir + file_name)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    img_id = df_image.iloc[i]['id']
    df_cb = df_annotation.loc[df_annotation['image_id'] == img_id][['category_id', 'bbox']]
    
    for i in range(len(df_cb)):
        
        x, y = list(map(int, df_cb['bbox'].iloc[i][0:2]))
        width, height = list(map(int, df_cb['bbox'].iloc[i][2:4]))
        name = df_categories.loc[df_cb['category_id'].iloc[i]]['name']
        B, G, R = list(map(int, df_categories.loc[df_cb['category_id'].iloc[i]]['color'].split(',')))
        img = cv2.rectangle(img, (x, y), (x+width, y+height), (B, G, R), 2)

    img2 = np.full((1024, 400, 3), 255, np.uint8)

    for i in range(10):
        point_text = 30, 100 + (i*85)
        point_color1 = 280, 100 + (i*85)
        point_color2 = 370, 130 + (i*85) - 50

        B, G, R = list(map(int, df_categories.iloc[i]['color'].split(',')))
        name = df_categories['name'].iloc[i] + ':'

        img2 = cv2.putText(img2, name, point_text, 0, 1, (0,0,0), 2)
        img2 = cv2.rectangle(img2, point_color1, point_color2, (B,G,R), -1)
    
    final_img = np.hstack((img, img2))
    
    _, file_name = file_name.split('/')
    #이미지 저장
    plt.imsave(f'/opt/ml/detection/dataset/new_annotated/{file_name}',final_img)

/opt/conda/envs/detection/lib/python3.7/site-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/4883 [00:00<?, ?it/s]